In [115]:
import os
import dspy
import orjson

In [116]:
lm = dspy.LM(
    model="groq/openai/gpt-oss-20b",
    api_key=os.environ.get("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)

dspy.configure(lm=lm)

In [230]:
import urllib.request

url = "https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_corpus.jsonl"
filename = "ragqa_arena_tech_corpus.jsonl"

# https://dspy.ai/tutorials/rag/#using-dspy-well-involves-evaluation-and-iterative-development

urllib.request.urlretrieve(url, filename)
print(f"Downloaded {filename}")

Downloaded ragqa_arena_tech_corpus.jsonl


In [118]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text-v2-moe:latest",
)

In [119]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path="ragqa_arena_tech_corpus.jsonl",
    jq_schema='[.doc_id, (.author // ""), .text] | join(" | ")',
    text_content=False,
    json_lines=True,
)

docs = loader.load()

In [120]:
docs[0]

Document(metadata={'source': 'C:\\Users\\mm0954\\Documents\\maventic_ai_101\\learn_dspy\\ragqa_arena_tech_corpus.jsonl', 'seq_num': 1}, page_content="229377 | Gilles 'SO- stop being evil' | The primary command to manipulate deb packages is dpkg-deb. To unpack the package, create an empty directory and switch to it, then run dpkg-deb to extract its control information and the package files. Use dpkg-deb -b to rebuild the package. mkdir tmp dpkg-deb -R original.deb tmp # edit DEBIAN/postinst dpkg-deb -b tmp fixed.deb Beware that unless your script is running as root, the files permissions and ownership will be corrupted at the extraction stage. One way to avoid this is to run your script under fakeroot. Note that you need to run the whole sequence under fakeroot, not each dpkg-deb individually, since its the fakeroot process that keeps the memory of the permissions of the files that cant be created as they are. fakeroot sh -c  mkdir tmp dpkg-deb -R original.deb tmp # edit DEBIAN/postinst

In [ ]:
embedder = dspy.Embedder(model="ollama/nomic-embed-text-v2-moe:latest")

In [122]:
embedder(["Hello, Hi!", "Hello, World!"])

array([[ 0.04890063,  0.05358162, -0.0017092 , ..., -0.04499949,
        -0.01059515,  0.08376051],
       [ 0.03208295,  0.01501115,  0.00023883, ..., -0.05685291,
        -0.02213773,  0.03984794]], shape=(2, 768), dtype=float32)

In [185]:
max_characters = 6000  # for truncating >99th percentile of documents

# with open("ragqa_arena_tech_corpus.jsonl") as f:
with open("temp.jsonl") as f:
    corpus = [orjson.loads(line)['text'][:max_characters] for line in f]
    print(f"Loaded {len(corpus)} documents. Will encode them below.")

Loaded 12 documents. Will encode them below.


In [186]:
embeddings = dspy.retrievers.Embeddings(corpus=corpus, embedder=embedder, k=2)
embeddings.save("./embeddings")

In [187]:
rm = dspy.retrievers.Embeddings(corpus=corpus, embedder=embedder)
rm.load(path="./embeddings", embedder=embedder)

In [188]:
dspy.configure(lm=lm, rm=rm)

In [ ]:
loaded_embeddings = dspy.Embeddings.from_saved("./embeddings", embedder)
loaded_embeddings.__call__("Silver quick")

2

In [190]:
max_characters = 6000

with open("embd_for_faiss.jsonl") as f:
    corpus = [orjson.loads(line)['text'][:max_characters] for line in f]
    print(f"Loaded {len(corpus)} documents. Will encode them below.")

Loaded 275 documents. Will encode them below.


In [191]:
embeddings = dspy.retrievers.Embeddings(corpus=corpus, embedder=embedder, k=2)
embeddings.save("./embeddings")

In [192]:
rm = dspy.retrievers.Embeddings(
    corpus=corpus, embedder=embedder)
rm.load(path="./embeddings", embedder=embedder)

In [193]:
dspy.configure(lm=lm, rm=rm)

In [194]:
loaded_embeddings("ayush singh")

Prediction(
    passages=['⇧+⌘+⌥+Q will perform a quick logout. Another option is to enable a password when waking from sleep or screen saver and adding a hot corner for one or the other. Then locking is as simple as tossing a mouse in a corner.', 'I think the answer to your question is No, there isnt a command line way to do this. Because this is a menu bar item, its not something that you can access easily using Keyboard Maestro or another similar tool. However, if you are not averse to a solution using a 3rd party app, QuickLock will let you do this. Its a free (donations accepted) app which will let you assign a keyboard shortcut to lock the screen. Theres also a menu bar item which you can click to lock the screen. The app will let you set a password (separate from your account password). It does not require the use of password with screensaver, its all completely separate. You can see a video of it in action at http://www.youtube.com/watch?v=SBHwykPB19o p.s. when the screen is lo

In [152]:
class RAG_QA(dspy.Signature):
    question: str = dspy.InputField(desc="Users query")
    context: list[str] = dspy.InputField(desc="Relavant context from the database")
    output: str = dspy.OutputField(desc="Reponse based on the Query and Context in 30 to 50 words")

In [155]:
prompt = "How to control the MAC remotely?"

context = loaded_embeddings(prompt).passages

prediction = dspy.Predict(RAG_QA)
prediction(question=prompt, context=context).output

'Use macOS built‑in features or third‑party tools. Enable **Screen Sharing** or **Apple Remote Desktop** in Sharing preferences, or connect via **SSH** for command‑line control. For remote lock/sleep, send the shortcut keys (⌥+⌘+⏏ or Control‑Command‑Q) from a remote session or via AppleScript.'

In [195]:
# lm.inspect_history(2)

In [183]:
dspy.configure(lm=lm, rm=rm)

In [214]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.retrievers.Embeddings(
            corpus=corpus, embedder=embedder, k=num_passages)
        self.generate_answer = dspy.Predict(RAG_QA)

    def forward(self, question):
        context = self.retrieve.__call__(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return prediction

In [216]:
question = "What is the name of the platform where I can use quick silver?"

rag = RAG(num_passages=2)
output = rag(question=question)
print(output.output)

The platform is macOS—QuickSilver is an application launcher for Mac computers, designed to remember and quickly launch your most frequently used apps. It’s a native macOS tool that streamlines launching with a few keystrokes.


In [218]:
# lm.inspect_history(2)